In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt
import sys
import numpy as np
import random
from utils import GroundTruthProcess
from utils import NCHW_to_NHWC_np
from utils import NHWC_to_NCHW_np
from DME_deformable import DMENet
from config import DefaultConfig
import torch.cuda as torch_cudab
import metrics
MAE = 10240000
%matplotlib inline

In [2]:
# obtain the gpu device
assert torch.cuda.is_available()
cuda_device = torch.device("cuda")  # device object representing GPU
opt = DefaultConfig()

In [3]:
# data_load
image_train = NHWC_to_NCHW_np(np.load(opt.train_data_path))
gt_train = NHWC_to_NCHW_np(np.load(opt.train_gt_path))
image_validate = NHWC_to_NCHW_np(np.load(opt.validate_data_path))
gt_validate = NHWC_to_NCHW_np(np.load(opt.validate_gt_path))

image_train_num = len(image_train)

In [4]:
# model construct
net = DMENet().to(cuda_device)
gt_map_process_model = GroundTruthProcess(1, 1, 8).to(cuda_device) # to keep the same resolution with the prediction

In [5]:
# set optimizer and estimator
criterion = metrics.DMELoss().to(cuda_device)
optimizer = torch.optim.Adam(net.parameters(), lr=opt.lr, weight_decay=opt.weight_decay)
ae_batch = metrics.AEBatch().to(cuda_device)
se_batch = metrics.SEBatch().to(cuda_device)

In [1]:
# train
for i in range(opt.max_epoch):
    shuffle_batch = np.random.permutation(image_train_num // opt.batch_size)
    step = 0
    for j in range(image_train_num // opt.batch_size):
        # validate
        net.eval()
        if step % opt.validate_steps == 0:
            loss_ = []
            MAE_ = []
            MSE_ = []
            
            # gather all validate information
            for k in range(len(image_validate // opt.batch_size)):
                if opt.use_gpu:
                    validate_x = torch.FloatTensor(image_validate[k:k + opt.batch_size]).cuda()
                    validate_gt = torch.FloatTensor(gt_validate[k:k + opt.batch_size]).cuda()
                validate_predict_map = net(validate_x)
                validate_gt_map = gt_map_process_model(validate_gt)
                # That’s because numpy doesn’t support CUDA, 
                # so there’s no way to make it use GPU memory without a copy to CPU first. 
                # Remember that .numpy() doesn’t do any copy, 
                # but returns an array that uses the same memory as the tensor
                validate_loss = criterion(validate_predict_map, validate_gt_map).data.cpu().numpy()
                batch_ae = ae_batch(validate_predict_map, validate_gt_map).data.cpu().numpy()
                batch_se = se_batch(validate_predict_map, validate_gt_map).data.cpu().numpy()
                loss_.append(validate_loss)
                MAE_.append(batch_ae)
                MSE_.append(batch_se)
            
            # calculate the validate loss, validate MAE and validate RMSE
            loss_ = np.reshape(loss_, [-1])
            MAE_ = np.reshape(MAE_, [-1])
            MSE_ = np.reshape(MSE_, [-1])
            
            validate_loss = np.mean(loss_)
            validate_MAE = np.mean(MAE_)
            validate_RMSE = np.sqrt(np.mean(MSE_))
            
            # show one sample from the validation
#             random_num = random.randint(0, 19)
            random_num = 1
            random_sample_x = torch.FloatTensor(image_validate[random_num:random_num+1]).cuda()
            random_sample_gt = torch.FloatTensor(gt_validate[random_num:random_num+1]).cuda()
            random_sample_predict = NCHW_to_NHWC_np(net(random_sample_x).data.cpu().numpy())
            random_sample_gt_map = NCHW_to_NHWC_np(gt_map_process_model(random_sample_gt).data.cpu().numpy())
            
            figure, (origin, density_gt, pred) = plt.subplots(1, 3, figsize=(20, 4))
            origin.imshow(np.squeeze(NCHW_to_NHWC_np(image_validate[random_num:random_num+1])))
            origin.set_title('Origin Image')
            density_gt.imshow(np.squeeze(random_sample_gt_map), cmap=plt.cm.jet)
            density_gt.set_title('ground_truth')
            pred.imshow(np.squeeze(random_sample_predict), cmap=plt.cm.jet)
            pred.set_title('back_end')
            plt.suptitle("one sample from the validate")
            plt.show()
            
            # show the validate MAE and MSE values on stdout
            gt_counts = np.squeeze(torch.sum(validate_gt_map, dim=(0, 1, 2, 3)).data.cpu().numpy())
            pred_counts = np.squeeze(torch.sum(validate_predict_map, dim=(0, 1, 2, 3)).data.cpu().numpy())
            sys.stdout.write('The gt counts of the above sample:{}, and the pred counts:{}\n'.format(gt_counts, pred_counts))
            sys.stdout.write('In step {}, epoch {}, with loss {}, MAE = {}, MSE = {}\n'.format(step, i + 1, validate_loss, validate_MAE, validate_RMSE))
            sys.stdout.flush()
            
            # save model
            if MAE > validate_MAE:
                MAE = validate_MAE
                torch.save(net, opt.model_save_path)
            
        
        # clear the gradient
        net.train()
        optimizer.zero_grad()
        # get the batch data
        start = (shuffle_batch[j] * opt.batch_size) % image_train_num
        end = min(start + opt.batch_size, image_train_num)
        if opt.use_gpu:
            x = torch.FloatTensor(image_train[start:end]).cuda()
            gt = torch.FloatTensor(gt_train[start:end]).cuda()
        # processs the gt because the final output only have 1/8 resolution
        gt_map = gt_map_process_model(gt)
        # flow the data through the net
        estimated_density_map = net(x)
        # calculate the loss and backward the gradient
        loss = criterion(estimated_density_map, gt_map)
        loss.backward()
        optimizer.step()
        step += 1
            
                
                

NameError: name 'opt' is not defined

In [23]:
a = torch.randn(2, 3)
b = torch.randn(2, 3)
print(a, b)
print

tensor([[ 0.4242,  0.7217,  1.1008],
        [-0.0442, -0.6967,  0.4862]]) tensor([[ 0.0191,  1.7691,  0.8167],
        [-1.4132, -0.5933,  0.9567]])
tensor([[0.0081, 1.2768, 0.8991],
        [0.0624, 0.4133, 0.4651]])
tensor([[0.1800, 0.5209, 1.2118],
        [0.0020, 0.4853, 0.2364]])
tensor(-3.2035)
tensor(-1.6017)


In [12]:
print(0.4807 * 0.4807)

0.23107249000000002


In [3]:
# optimizer = torch.optim.Adam()
print(len(list(net.parameters())))

71


In [16]:
a = list(list(net.children())[1])[0]
b = list(list(a.children())[0].children())[2]

In [8]:
x = np.random.rand(1, 3, 4 ,5)
print(x, x.shape)
x = np.swapaxes(x, 1, 2)
x = np.swapaxes(x, 2, 3)
print(x, x.shape)

[[[[0.56534958 0.24370027 0.72369011 0.67889216 0.04249182]
   [0.00842371 0.48885883 0.21877356 0.75751709 0.55534385]
   [0.21609105 0.38360997 0.69587199 0.57972236 0.37004643]
   [0.77530353 0.21747802 0.84950338 0.1185008  0.06807957]]

  [[0.00871294 0.36645846 0.45530013 0.104406   0.56377999]
   [0.48795602 0.76807449 0.06316015 0.01205147 0.9723964 ]
   [0.23015443 0.01537153 0.98212864 0.04200032 0.20598123]
   [0.76175589 0.60508917 0.8725611  0.8562562  0.02366146]]

  [[0.6246715  0.16085285 0.43329321 0.18251999 0.04658025]
   [0.83781238 0.86846604 0.46351249 0.16919308 0.50824695]
   [0.35417069 0.84248133 0.80769374 0.41581033 0.23163003]
   [0.81354884 0.64415368 0.27507783 0.01011959 0.18384952]]]] (1, 3, 4, 5)
[[[[0.56534958 0.00871294 0.6246715 ]
   [0.24370027 0.36645846 0.16085285]
   [0.72369011 0.45530013 0.43329321]
   [0.67889216 0.104406   0.18251999]
   [0.04249182 0.56377999 0.04658025]]

  [[0.00842371 0.48795602 0.83781238]
   [0.48885883 0.76807449 0.86